In [1]:
from IslandTime import retrieve_island_info, PreTimeSeries, update_data_map, Workflow, TimeSeriesCoastSat, TimeSeriesERA5, TimeSeriesClimateIndices, TimeSeriesSeaLevelAnomaly, check_last_date_download
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium
import webbrowser
import shapely
import matplotlib
import datetime
%matplotlib qt

import osmnx as ox
%load_ext autoreload
%autoreload 2

In [4]:
island = 'Bodu' # Vakkaru
country = 'Maldives'

# check_last_date_download(island, country)
# 
# island_info = retrieve_island_info(island, country)

# TimeSeriesCoastSat(island, country, reference_shoreline_transects_only=True, retrieve_reference_shoreline_manually=True, overwrite=True).main()
# TimeSeriesERA5(island, country).main()
# TimeSeriesClimateIndices(island, country).main()
# TimeSeriesSeaLevelAnomaly(island, country).main()

ii = Workflow(island, country, run_all=False, execute_segmentation=False, execute_preprocess=False, execute_analysis=True, small_island=False).main()


-------------------------------------------------------------------
Retrieving all information available for the island
Island: Bodu, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               atoll
spatial_reference
               latitude
               longitude
               polygon
               polygon_OSM
               reference_shoreline
               transects
               area_country
               reference_shoreline_buffer_L8
               reference_shoreline_buffer_L9
               reference_shoreline_buffer_S2
image_collection_dict
               description
               S2
               L5
               L7
               L8
               L9
timeseries_CRW
               description
               description_timeseries
               source
               timeseries
timeseries_nighttime_light
               description
  

In [6]:
update_data_map()

Updating the map with the progress of the islands...


  0%|          | 1/614 [00:00<04:01,  2.54it/s]

Aakiraahuttaa


  6%|▌         | 35/614 [00:12<02:26,  3.95it/s]

In [11]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add nodes (variables)
G.add_node("coastline_position")
G.add_node("wave_ucomp")
G.add_node("wave_vcomp")
G.add_node("sea_level")

# Add edges (causal relationships)
G.add_edges_from([
    ("wave_ucomp", "coastline_position"),                   # IOD influences SST
    ("sea_level", "wave_ucomp"),        # SST influences 2m_temperature
    ("sea_level", "coastline_position"), 
    ("wave_vcomp", "wave_ucomp")           # SST influences wind_u10            # SST influences wind_v10     # Wind influences evaporation
])

# Draw the graph
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, seed=42)  # positions for all nodes
nx.draw(G, pos, with_labels=True, node_color='lightgreen', node_size=3500, font_size=10, font_weight='bold', arrows=True)
plt.title('Causal Graph of Variables with Precipitation and Evaporation', fontsize=15)
plt.show()


In [8]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add nodes (variables)
G.add_node("wind_u10")
G.add_node("wind_v10")
G.add_node("2m_temperature")
G.add_node("SST")
G.add_node("IOD")
G.add_node("MSLP") 
G.add_node("total_precipitation")
G.add_node("evaporation")

# Add edges (causal relationships)
G.add_edges_from([
    ("IOD", "SST"),                   # IOD influences SST
    ("SST", "2m_temperature"),        # SST influences 2m_temperature
    ("SST", "wind_u10"),              # SST influences wind_u10
    ("SST", "wind_v10"),              # SST influences wind_v10
    ("SST", "MSLP"),                  # SST influences MSLP
    ("SST", "evaporation"),           # SST influences evaporation
    ("IOD", "wind_u10"),              # IOD influences wind_u10
    ("IOD", "wind_v10"),              # IOD influences wind_v10
    ("IOD", "2m_temperature"),        # IOD influences 2m_temperature
    ("IOD", "MSLP"),                  # IOD influences MSLP
    ("2m_temperature", "wind_u10"),   # 2m_temperature influences wind_u10
    ("2m_temperature", "wind_v10"),   # 2m_temperature influences wind_v10
    ("2m_temperature", "evaporation"),# 2m_temperature influences evaporation
    ("wind_u10", "SST"),              # wind_u10 influences SST
    ("wind_v10", "SST"),              # wind_v10 influences SST
    ("wind_u10", "wind_v10"),         # wind_u10 influences wind_v10 (and vice versa)
    ("wind_v10", "wind_u10"),
    ("MSLP", "wind_u10"),             # MSLP influences wind_u10
    ("MSLP", "wind_v10"),             # MSLP influences wind_v10
    ("MSLP", "IOD"),                  # MSLP influences IOD
    ("SST", "IOD"),                   # SST influences IOD
    ("evaporation", "total_precipitation"), # Evaporation influences precipitation
    ("MSLP", "total_precipitation"),  # MSLP influences precipitation
    ("total_precipitation", "2m_temperature"), # Precipitation influences temperature
    ("wind_u10", "evaporation"),      # Wind influences evaporation
    ("wind_v10", "evaporation"),      # Wind influences evaporation
])

# Draw the graph
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, seed=42)  # positions for all nodes
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=3500, font_size=10, font_weight='bold', arrows=True)
plt.title('Causal Graph of Variables with Precipitation and Evaporation', fontsize=15)
plt.show()



In [35]:
for island in ["Hulhidhoo (Felidhu)"]:
    west, north, east, south = 3.364167, 73.497272, 3.358160, 73.503410


    polygon_array = [[north, west],
                      [south, west],
                        [south, east],
                          [north, east],
                            [north, west]]
    # 
    PreTimeSeries(island, 'Maldives', atoll='Muli').main()
    # PreTimeSeries(island, 'Maldives', atoll='Felidhu', polygon=polygon_array).main()


-------------------------------------------------------------------
Retrieving general and spatial information about the island
Island: Hulhidhoo (Felidhu), Maldives
-------------------------------------------------------------------

~ All information will be extracted/calculated. ~
S2 70
L5 0
L7 29
L8 67
L9 24
~ Retrieving information from Duvat & Magnan (2019). ~
Island not found in the database.


In [33]:
island_info = retrieve_island_info('Ruh hurihuraa', 'Maldives', verbose=False)
island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']

(3.361383, 73.500085)

In [49]:
island_info['timeseries_climate_indices']['timeseries'].columns

Index(['Dipole Mode Index (Indian Ocean Dipole Index)',
       'Pacific North American Index',
       'East Pacific/North Pacific Oscillation', 'Western Pacific Index',
       'Eastern Atlantic/Western Russia', 'North Atlantic Oscillation',
       'North Atlantic Oscillation (Jones)', 'Southern Oscillation Index',
       'Eastern Tropical Pacific SST (anomalies)',
       'Eastern Tropical Pacific SST (mean values)',
       'Bivariate ENSO Timeseries', 'Tropical Northern Atlantic Index',
       'Tropical Southern Atlantic Index', 'Western Hemisphere Warm Pool',
       'Oceanic Niño Index', 'Multivariate ENSO Index (MEI V2)',
       'Extreme Eastern Tropical Pacific SST (anomalies)',
       'Extreme Eastern Tropical Pacific SST (mean values)',
       'Central Tropical Pacific SST (anomalies)',
       'Central Tropical Pacific SST (mean values)',
       'East Central Tropical Pacific SST (anomalies)',
       'East Central Tropical Pacific SST (mean values)',
       'Pacific Decadal Oscill

In [52]:
island_info['timeseries_climate_indices']['timeseries']['Multivariate ENSO Index (MEI V2)'].plot()

<Axes: >

In [1]:
import matplotlib.pyplot as plt
import Rbeast as rb
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

%load_ext autoreload
%autoreload 2

In [34]:
tags = {'place':'islet', 'natural':'coastline'}
gdf = ox.features_from_place("Faadhippolhu", tags=tags)

In [35]:
gdf['name:en'].way.values

array([nan, nan, 'Maabinhuraa', nan, 'Olhuvelifushi', nan,
       'Thilamaafushi', 'Fainuaadhamhuraa', nan, nan, nan, 'Innahura',
       nan, nan, nan, nan, 'Raiyruhhuraa', nan, 'Mayyaafushi', nan,
       'Faadhoo', nan, nan, nan, 'Meedhaahuraa', 'Kurendhoo', nan,
       'Maafilaafushi', 'Lhohi', 'Lhossalafushi', 'Aligaa', 'Medhafushi',
       'Varihuraa', nan, nan, 'Madivaru', 'Vavvaru', nan, 'Felivaru',
       'Veyvah', nan, nan, 'Naifaru', nan, nan, nan, nan, 'Musleygihuraa',
       nan, nan, 'Medhadhihuraa', nan, 'Gaavelifaru', 'Hinnavaru'],
      dtype=object)

In [ ]:
islands = gdf['name:en'].way.values
for island in islands:
    if type(island) == str:
        try:
            PreTimeSeries(island, 'Maldives').main()
        except:
            continue

In [2]:
path_to_data = os.path.join(os.getcwd(), 'data', 'info_islands')

latitude, longitude, colors, names, desc = [], [], [], [], []

for file in os.listdir(path_to_data):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]
    island_info = retrieve_island_info(island, country, verbose=False)
    latitude.append(island_info['spatial_reference']['latitude'])
    longitude.append(island_info['spatial_reference']['longitude'])
    names.append(island_info['general_info']['island'])

    if 'visual_inspection' in island_info['general_info'].keys():
        colors.append('blue')
        desc.append('Visual inspection done')
    
    else:
        if 'timeseries_coastsat' in island_info.keys():
            if 'timeseries' in island_info['timeseries_coastsat'].keys():
                colors.append('green')
                desc.append('CoastSat time series available')
            
            else:
                if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                    colors.append('orange')
                    desc.append('CoastSat images are being downloaded')
        
        else:
            if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                colors.append('orange')
                desc.append('CoastSat images are being downloaded')
            
            else:
                colors.append('red')
                desc.append('CoastSat has not started downloading images yet')

# Create a base map
m = folium.Map(location=[sum(latitude) / len(latitude), sum(longitude) / len(longitude)], zoom_start=5)

# Add markers for each point
for lat, lon, color, name, des in zip(latitude, longitude, colors, names, desc):
    folium.Marker([lat, lon], popup=str(name)+'\n'+str(des), icon=folium.Icon(color=color)).add_to(m)

# Save the map to an HTML file
m.save('progress_island_mapping.html')

# Open the map in a web browser
webbrowser.open('progress_island_mapping.html')

True

In [28]:
import pyproj
from bs4 import BeautifulSoup

island = 'Vodamulaa'
country = 'Maldives'

island_info = retrieve_island_info(island, country, verbose=False)
reference_shoreline = island_info['spatial_reference']['reference_shoreline']
transects = island_info['spatial_reference']['transects']
polygon_reference_shoreline = shapely.geometry.Polygon(reference_shoreline)

latitude, longitude = island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']
ts_analysis_results = island_info['timeseries_analysis']
key_transects = [int((key).split('_')[3]) for key in ts_analysis_results.keys()]
intersections = [polygon_reference_shoreline.exterior.intersection(shapely.geometry.LineString(transects[key_transect])) for key_transect in key_transects]
x_intersections = [intersection.x for intersection in intersections]
y_intersections = [intersection.y for intersection in intersections]

m = folium.Map(location=[latitude, longitude], zoom_start=15)

# Reproject the coordinates to the Web Mercator projection
# Source and target coordinate reference systems
tgt_crs = pyproj.CRS('EPSG:4326')
src_crs = pyproj.CRS('EPSG:3857')

# Define transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# Transform latitude and longitude to x and y
x_intersections, y_intersections = transformer.transform(x_intersections, y_intersections)

# Transform reference shoreline to x and y
x_reference_shoreline, y_reference_shoreline = transformer.transform(polygon_reference_shoreline.exterior.coords.xy[0], polygon_reference_shoreline.exterior.coords.xy[1])

c_trend = [ts_analysis_results[val]['trend']['trend_slope'] for val in ts_analysis_results.keys()]
symbols_trend = [ts_analysis_results[val]['trend']['trend_result'] for val in ts_analysis_results.keys()]

# Add markers for each point with different shapes
for lat, lon, symbol in zip(y_intersections, x_intersections, symbols_trend):
    if symbol == "increasing":
        icon = folium.Icon(color='green', icon='arrow-up')
    elif symbol == "decreasing":
        icon = folium.Icon(color='red', icon='arrow-down')
    elif symbol == "no trend":
        icon = folium.Icon(color='orange', icon='minus')
    else:
        icon = folium.Icon(color='black', icon='question-sign')

    folium.Marker(
        location=[lat, lon],
        icon=icon
    ).add_to(m)

folium.PolyLine(np.dstack((y_reference_shoreline, x_reference_shoreline)), tooltip="Coast", color='black').add_to(m)

# Save the map to an HTML file
m.save('results_islands_mapping.html')

# Open the map in a web browser
webbrowser.open('results_islands_mapping.html')

True